In [1]:
from pystac_client import Client
import planetary_computer as pc

# Search against the Planetary Computer STAC API
catalog = Client.open(
  "https://planetarycomputer.microsoft.com/api/stac/v1"
)

# Define your area of interest
aoi = {
  "type": "Polygon",
  "coordinates": [
    [
      [-56.18613082072048, -11.496416488275614],
      [-54.88756817943009, -11.496416488275614],
      [-54.88756817943009, -10.769082694296898],
      [-56.18613082072048, -10.769082694296898],
      [-56.18613082072048, -11.496416488275614]
    ]
  ]
}

# Define your temporal range
daterange = {"interval": ["2022-05-01", "2022-05-31T23:59:59Z"]}

# Define your search with CQL2 syntax
search = catalog.search(filter_lang="cql2-json", filter={
  "op": "and",
  "args": [
    {"op": "s_intersects", "args": [{"property": "geometry"}, aoi]},
    {"op": "anyinteracts", "args": [{"property": "datetime"}, daterange]},
    {"op": "=", "args": [{"property": "collection"}, "hls2-s30"]},
    {"op": "<=", "args": [{"property": "eo:cloud_cover"}, 10]}
  ]
})

# Grab the first item from the search results and sign the assets
first_item = next(search.get_items())
pc.sign_item(first_item).assets


APIError: The request exceeded the maximum allowed time, please try again. If the issue persists, please contact planetarycomputer@microsoft.com.



In [ ]:
import IPython.display as disp
disp.Image(url=first_item.assets["thumbnail"].href)


In [2]:
from pystac_client import Client
import planetary_computer as pc
import leafmap

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

aoi = {
    "type": "Polygon",
    "coordinates": [[
        [-56.1861, -11.4964],
        [-54.8875, -11.4964],
        [-54.8875, -10.7690],
        [-56.1861, -10.7690],
        [-56.1861, -11.4964]
    ]]
}

# ✅ 改成 sentinel-2-l2a 集合
search = catalog.search(
    collections=["sentinel-2-l2a"],
    datetime="2022-05-01/2022-05-31",
    query={"eo:cloud_cover": {"lt": 10}},
    intersects=aoi,
)

item = next(search.get_items())
item = pc.sign_item(item)

# ✅ 直接取出 True Color 资产
rgb_url = item.assets["visual"].href

m = leafmap.Map(center=[-11.1, -55.5], zoom=9)
m.add_cog_layer(url=rgb_url, name="Sentinel-2 True Color", attribution="MPC Sentinel-2 L2A")
m

APIError: The request exceeded the maximum allowed time, please try again. If the issue persists, please contact planetarycomputer@microsoft.com.



In [9]:
from pystac_client import Client
import planetary_computer as pc
import leafmap
from shapely.geometry import shape

catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")

search = catalog.search(
    collections=["sentinel-2-l2a"],
    datetime="2022-05-01/2022-05-31",
    query={"eo:cloud_cover": {"lt": 10}},
    intersects=aoi,
)

item = next(search.get_items())
item = pc.sign_item(item)

# Get item ID
item_id = item.id

# Build false-color tile URL
falsecolor_url = (
    f"https://planetarycomputer.microsoft.com/api/data/v1/item/tiles/WebMercatorQuad/{{z}}/{{x}}/{{y}}@1x"
    f"?collection=sentinel-2-l2a"
    f"&item={item_id}"
    f"&assets=B08&assets=B04&assets=B03"
    f"&color_formula=gamma+RGB+1.8,saturation+1.5"
    f"&format=png"
)

geom = shape(item.geometry)
m = leafmap.Map(center=[geom.centroid.y, geom.centroid.x], zoom=9)
m.add_tile_layer(url=falsecolor_url, name="Sentinel-2 False Color", attribution="MPC Sentinel-2 L2A")
m.add_basemap("SATELLITE")
m

Map(center=[-10.438779728241167, -54.67200547579291], controls=(ZoomControl(options=['position', 'zoom_in_text…

In [8]:
print(item.assets.keys())

dict_keys(['AOT', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B11', 'B12', 'B8A', 'SCL', 'WVP', 'visual', 'safe-manifest', 'granule-metadata', 'inspire-metadata', 'product-metadata', 'datastrip-metadata', 'tilejson', 'rendered_preview'])


In [1]:
import geopandas as gpd
import leafmap
from shapely.geometry import mapping

# --- load shapefile ---
points = gpd.read_file("/usr2/postdoc/chishan/project_data/MapBiomas/MAPBIOMAS/mapbiomas_amazon_sample_100.shp")   # ensure it’s in EPSG:4326 (lon/lat)
points = points.to_crs(epsg=4326)

# --- create map ---
m = leafmap.Map(center=[points.geometry.y.mean(), points.geometry.x.mean()], zoom=8)
m.add_basemap("SATELLITE")

# --- add points to map ---
m.add_gdf(points, layer_name="Sample Points")

# m


In [3]:
import geopandas as gpd
import leafmap
from pystac_client import Client
import planetary_computer as pc
from shapely.geometry import mapping

# --- 1. Load your shapefile (points) ---
points = gpd.read_file("/usr2/postdoc/chishan/project_data/MapBiomas/MAPBIOMAS/mapbiomas_amazon_sample_100.shp")   # ensure it’s in EPSG:4326 (lon/lat)

# --- 2. Create interactive map ---
m = leafmap.Map(center=[points.geometry.y.mean(), points.geometry.x.mean()], zoom=8)
m.add_basemap("SATELLITE")
m.add_gdf(points, layer_name="Sample Points")

# --- 3. Connect to Planetary Computer ---
catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")


# # --- 4. Define click callback ---
# def handle_click(**kwargs):
#     if kwargs.get("type") == "click":
#         lon, lat = kwargs["coordinates"]
#         load_sentinel2_image(lon, lat)

# # --- 5. Bind callback to map ---
# m.on_interaction(handle_click)

m


Map(center=[np.float64(-8.8214386693369), np.float64(-53.25376540326599)], controls=(ZoomControl(options=['pos…

In [6]:
print(item.assets.keys())

NameError: name 'item' is not defined

In [5]:
def load_sentinel2_image(lon, lat):
    """Fetch one Sentinel-2 true-color image near a clicked point."""
    aoi = {
        "type": "Point",
        "coordinates": [lon, lat]
    }

    # STAC search (here example: June 2022, cloud < 10%)
    search = catalog.search(
        collections=["sentinel-2-l2a"],
        intersects=aoi,
        datetime="2022-06-01/2022-06-30",
        query={"eo:cloud_cover": {"lt": 10}},
    )

    try:
        # Take first match, sign URL
        item = next(search.get_items())
        item = pc.sign_item(item)
        rgb_url = item.assets["visual"].href

        # --- remove old Sentinel-2 layers first ---
        for layer in list(m.layers):
            if "Sentinel-2" in layer.name:
                m.remove_layer(layer)

        # --- add the new image layer ---
        m.add_cog_layer(
            url=rgb_url,
            name=f"Sentinel-2 {item.id}",
            attribution="Planetary Computer Sentinel-2",
            opacity=0.85,
        )
        print(f"Loaded {item.id}")

    except StopIteration:
        print("No Sentinel-2 image found here.")
        
load_sentinel2_image(-59.11871, -15.75753)

Loaded S2B_MSIL2A_20220628T140709_R110_T21LTC_20220629T045154


In [4]:
points

,Name,Descriptio,sample_id,geometry
0,Point 2414584 - Year 2013.0,None,1,POINT Z (-49.96883 -12.67316 0)
1,Point 2334927 - Year 2012.0,None,2,POINT Z (-49.74191 -6.95314 0)
2,Point 2496730 - Year 2014.0,None,3,POINT Z (-46.43252 -12.353 0)
3,Point 2229257 - Year 2011.0,None,4,POINT Z (-62.48146 -11.11925 0)
4,Point 2873836 - Year 2018.0,None,5,POINT Z (-47.10976 -1.38134 0)
...,...,...,...,...
95,Point 2657882 - Year 2016.0,None,96,POINT Z (-59.11871 -15.75753 0)
96,Point 2420223 - Year 2013.0,None,97,POINT Z (-49.51015 -7.54414 0)
97,Point 2501737 - Year 2014.0,None,98,POINT Z (-55.7856 -12.6613 0)
98,Point 2762609 - Year 2017.0,None,99,POINT Z (-45.16455 -4.37003 0)


In [1]:
import leafmap
from pystac_client import Client
import planetary_computer as pc

# --- 1. 创建地图 ---
m = leafmap.Map(center=[-11.1, -55.5], zoom=8)
m.add_basemap("SATELLITE")

# --- 2. 定义点击事件：只输出经纬度 ---
def handle_click(**kwargs):
    if kwargs.get("type") == "click":
        lon, lat = kwargs["coordinates"]
        print(f"Clicked location: lon={lon:.5f}, lat={lat:.5f}")

m.on_interaction(handle_click)
m


Map(center=[-11.1, -55.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_o…

In [ ]:
m
